<a href="https://colab.research.google.com/github/woshidama323/DATAREPO/blob/master/juliaforcolaboncolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 首先利用python3 安装julia环境 
## 安装最新的julia的程序
!curl -sSL https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz -o julia.tar.gz

!tar -xzf julia.tar.gz -C /usr --strip-components 1
!julia --version
## 安装IJulia来安装jupyter
!julia -e 'using Pkg; pkg"add IJulia;"'


In [ ]:
### 创建ssh 连接到虚拟机 要用python3环境
## 启动ssh登录方式
# # Install colab_ssh
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh, init_git
launch_ssh("1bSjkXryWB907qK8Jzt6ZkvgpdI_3FYxy4R51a9u1ws9ZWQXm","hello123")

# # Optional: if you want to clone a github repository
# ## 同步自己的github repo
# init_git("https://github.com/woshidama323/JuliaBoxSync.git")

## 新版本
# Install colab_ssh on google colab
# !pip install colab_ssh --upgrade

# from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
# launch_ssh_cloudflared(password="hellotest")

### 这种方式目前不成功，原因是卡在cloudflare中 这里不太会用 有空再研究吧

In [ ]:
### 切到julia1.5环境，统一安装julia库
using Pkg
Pkg.add("Plots")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Cairo")
Pkg.add("Fontconfig")
Pkg.add("Plotly")
Pkg.add("Gadfly")

In [ ]:
;cd /content/JuliaBoxSync

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
using CSV,DataFrames,Gadfly#Plots

basefee = CSV.read("/content/JuliaBoxSync/basefee.0224",DataFrame,header=false)

feemax = maximum(basefee[!, :Column2])
feemin = minimum(basefee[!, :Column2])

## 写到文件中的方法
#plot(1:length(basefee[!, :Column1]),basefee[!, :Column2],fmt=:svg)
#savefig("basefee0224.svg")

## draw 一个可交互式的plot图
first(basefee,100);
##注意这里可能之前plots 包已经进入了，这里直接调用plot的话是用的Plots中的plot 而不是Gadfly中的plot
#p = Gadfly.plot(basefee, x=1:length(basefee.Column1), y=:Column2, hover=data,Geom.line)
 p = Gadfly.plot(basefee, x=1:length(basefee.Column1), y=:Column2, hover=data, Geom.point, Geom.smooth(method=:loess,smoothing=0.01))
 
  #p = Gadfly.plot(basefee, x=1:length(basefee.Column1), y=:Column2, hover=data,Geom.subplot_grid(Geom.bar))
#img = SVG("iris_plot.svg", 8cm, 4cm)
#draw(img, p)

##画一个可以交互式的图
##但是这里的图怎么显示在jupyter中
draw(SVGJS("foo.svg", 50cm, 20cm), p)


In [ ]:
Pkg.add("Gadfly")
Pkg.add("RDatasets")
using Gadfly, RDatasets
@show iris = dataset("datasets", "iris")

p = Gadfly.plot(iris, x=:SepalLength, y=:SepalWidth, Geom.point)



In [ ]:
using Plots
plotly()
data = 1:5
plt = Plots.plot(data; hover=data )
display("text/html", plt)

In [ ]:
;git status

In [ ]:
;git add .

In [ ]:
;git commit -m "update the code"

In [ ]:
;git push

--------------------------------------------------------------------------------PYTHON3 --------------------------------------------------------------------------------

In [ ]:
### python环境下的basefee的图
## 文件位置/content/JuliaBoxSync/basefee.0224
import csv
with open("/content/JuliaBoxSync/basefee.0224") as csvfile:
  spanreader = csv.reader(csvfile,delimiter = ' ',)
  for row in spanreader:
    print(', '.join(row))

In [ ]:
## 直接用python的方法来完成这次数据分析吧
## 分析的目的是什么？ 看看当前的数据是否有统计意义，什么样的关系
import psycopg3,os
import matplotlib.pyplot as plt
os.environ['PGHOST'] = 'localhost'

# Connect to an existing database
with psycopg3.connect("dbname=postgres user=postgres port=54322") as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        cur.execute("""
            select * from trade where base = 'BTC' order by tid limit 100
            """)
          
        fetchall = cur.fetchall()
        print(len(fetchall))
        for i in fetchall:
          print(i)

        btcpricelist = [i[6] for i in fetchall]
        plt.plot(btcpricelist)
        plt.show()

        # cur.execute("""
        #     CREATE TABLE test (
        #         id serial PRIMARY KEY,
        #         num integer,
        #         data text)
        #     """)

        # Pass data to fill a query placeholders and let Psycopg perform
        # the correct conversion (no SQL injections!)
        # cur.execute(
        #     "INSERT INTO test (num, data) VALUES (%s, %s)",
        #     (100, "abc'def"))

        # # Query the database and obtain data as Python objects.
        # cur.execute("SELECT * FROM test")
        # cur.fetchone()
        # # will return (1, 100, "abc'def")

        # # You can use `cur.fetchmany()`, `cur.fetchall()` to return a list
        # # of several records, or even iterate on the cursor
        # for record in cur:
        #     print(record)

        # # Make the changes to the database persistent
        # conn.commit()


In [ ]:
### 安装依赖包
!pip install git+https://github.com/psycopg/psycopg3.git#subdirectory=psycopg3